In [2]:
from IPython.display import HTML, display

# Results

We ran our experiment 5 times for each method and reported the best one.
Here our methods are TCN, GRU, GRU-bidirection, LSTM and LSTM-bidirection. Our methods are reported on both BERT and Glove language models. In addition, for the comparison, we have included the top 5 models and baseline methods from the competition as well.
The methods were sorted based on F1 score in the competition. We also followed this convention for our methods.

In [30]:
data = [['method', 'acc', 'precision', 'recall', 'F1'],
        ['THU_NGN', '0.735', '0.630', '0.801', '0.705'],
        ['NTUA-SLP', '0.732' , '0.654', '0.691', '0.672'],
        
        ['WLV' ,'0.643', '0.532', '0.836', '0.650'],
        
        ['NLRPL-IITBHU', '0.661', '0.551', '0.788', '0.648'],
        
        ['NIHRIO', '0.702', '0.609', '0.691', '0.648'],
        
        ['GRU_BERT', '0.737', '0.653', '0.702' ,'0.685'],
        ['Bi_GRU_BERT', '0.731' ,'0.663', '0.563', '0.658'],
        ['LSTM_BERT' ,'0.739' , '0.702', '0.591' , '0.642'],
        ['TCN_BERT' ,'0.673' , '0.571' , '0.714' , '0.634'],
        ['Bi_LSTM_BERT', '0.726', '0.692', '0.556', '0.617'],
        
        ['GRU_Glove', '0.666', '0.562', '0.711', '0.628'],
        ['LSTM_Glove', '0.682', '0.591', '0.650', '0.619'],
        ['Bi_GRU_Glove', '0.681', '0.588' ,'0.653', '0.619'],
        ['Bi_LSTM_Glove', '0.684', '0.603', '0.592' ,'0.597'],
        ['TCN_Glove', '0.634', '0.531', '0.662', '0.589'],
        ['Unigram SVM', '0.635', '0.532', '0.659', '0.589'],
        ['Random', '0.503', '0.373', '0.373', '0.373'],
       ]
display(HTML(
    #'<html><head><style>table, th, td {border: 1px solid black;}</style></head><body>'
   "<table border = 1> <tr>{}</tr></table>".format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
       )
))

method,acc,precision,recall,F1
THU_NGN,0.735,0.630,0.801,0.705
NTUA-SLP,0.732,0.654,0.691,0.672
WLV,0.643,0.532,0.836,0.650
NLRPL-IITBHU,0.661,0.551,0.788,0.648
NIHRIO,0.702,0.609,0.691,0.648
GRU_BERT,0.737,0.653,0.702,0.685
Bi_GRU_BERT,0.731,0.663,0.563,0.658
LSTM_BERT,0.739,0.702,0.591,0.642
TCN_BERT,0.673,0.571,0.714,0.634
Bi_LSTM_BERT,0.726,0.692,0.556,0.617


# Analysis of the Results

As it is shown in the table above, our GRU and LSTM models based on BERT language model have achieved the best accuracy among the competition models. Moreover, we have the best precision with LSTM on BERT language model. For the recall and F1 score we didn't beat the best model, but we have achieved the F1 score to get the second place in the competition.
The reason for the lower F1 score could be having low recall score. It seems that achieving better recall values would be feasible with some parameter tuning, which we didn't have so much time to do it perfectly.

# Future Work

In this project we used pre-trained embeddings and compared TCN and some variations of RNN models for detecting irony tweets. For the future we have some ideas to augment our data by adding or removing some unimportant words. We can also use attention to specify the importance of hidden units we use to generate the output. We could also concatenate embeddings from different langauge models to capture differnt embeddings in various contexts and types of sentences, but it needs a good tokenizer that can work for different language models, which seems hard and time consuming, and we didn't have the oppurtunity to check this direction.At last, as mentioned, parameter fine-tuning is also a good method to improve the performance, but again we didn't have so much time to investigate it completely.
